In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.dates as mdates
import sys
import plotly.graph_objs as go
from plotly.subplots import make_subplots
pd.options.mode.chained_assignment = None

In [ ]:
atokens_df = pd.read_csv("/content/drive/MyDrive/aave_files/aave_atokens_v2.csv")
user_df = pd.read_csv("/content/drive/MyDrive/aave_files/aave_liquidation_loss_v2.csv").sort_values(by='debtLeft (in USD)', ascending=False).reset_index(drop=True).head(10)
lending_pool_df = pd.read_csv("/content/drive/MyDrive/aave_files/lending_pool_logs_v2.csv", low_memory=False)

In [ ]:
#Mapping token addresses to symbols
tokens = { r['address']: r['symbol'] for _, r in atokens_df.iterrows() }

#Getting decimals for each token
decimals = { r['symbol']: r['decimals'] for _, r in atokens_df.iterrows() }

In [ ]:
ctr = 0
for address, symbol in tokens.items():
    if ctr == 0:
        transfers_df = pd.read_csv(f"/content/drive/MyDrive/aave_files/atoken_transfers/{symbol}_atoken_transfers.csv", low_memory=False)
        transfers_df['reserve'] = address
        transfers_df = transfers_df[transfers_df['event'] == 'Transfer']
    else:
        temp_df = pd.read_csv(f"/content/drive/MyDrive/aave_files/atoken_transfers/{symbol}_atoken_transfers.csv", low_memory=False)
        temp_df['reserve'] = address
        transfers_df = pd.concat( [transfers_df,temp_df] )
        transfers_df = transfers_df[transfers_df['event'] == 'Transfer']
    ctr += 1

In [ ]:
transfers_df = transfers_df[transfers_df['from'] != '0x0000000000000000000000000000000000000000']
transfers_df = transfers_df[transfers_df['to'] != '0x0000000000000000000000000000000000000000']

#Dropping and adding columns from the 2 dataframes, to make the column names the same, so we can merge them into one dataframe for Deposit, Withdraw and Transfer events
transfers_df.drop(['msg.sender', 'owner', 'spender'], axis=1, inplace = True)
transfers_df = transfers_df.reindex(columns = transfers_df.columns.tolist() + ['user', 'onBehalfOf', 'amount', 'collateralAsset','debtAsset','debtToCover','liquidatedCollateralAmount'])
df = lending_pool_df.drop(['liquidityRate','stableBorrowRate','variableBorrowRate','liquidityIndex','variableBorrowIndex','msg.sender','referral','to','borrowRateMode','borrowRate','repayer','rateMode','target','initiator','asset','premium','referralCode','liquidator','receiveAToken'], axis=1)
df = df.reindex(columns = df.columns.tolist() + ['from', 'to', 'value'])
new_lending_pool_df = pd.concat( [df,transfers_df] )

new_lending_pool_df.sort_values(by=['timestamp'], inplace=True)
new_lending_pool_df = new_lending_pool_df.reset_index(drop = True)

In [ ]:
#Function to get the price of an asset closest to a particular timestamp
def get_usd_price(address, timestamp):
    new_usd_df = pd.read_csv(f"/content/drive/MyDrive/aave_files/usd_prices/{tokens[address]}_usd.csv")
    new_usd_df['timestamp'] = new_usd_df['timestamp']//1000
    differences = abs(new_usd_df['timestamp'] - timestamp)
    min_diff_index = differences.idxmin()
    closest_value = new_usd_df.loc[min_diff_index, 'USD_price']
    return closest_value

In [132]:
#Function to plot events emitted by the Lending Pool
def add_event_lines(fig, event_timestamps, color, label, row=None, col=None):
    if label in ['Deposits', 'Withdrawals', 'Transfer In', 'Transfer Out', 'Repays', 'Reserve Enabled as Collateral', 'Reserve Disabled as Collateral']:
        show_legend = True  # Only show legend for the first line
        for ts, reserve, amount in event_timestamps:
            token = tokens[reserve]
            if label not in ['Reserve Enabled as Collateral', 'Reserve Disabled as Collateral']:
                if label in ['Deposits', 'Withdrawals', 'Repays']:
                    amount = (amount * get_usd_price(reserve, ts)) / (10**decimals[token])
                amount = round(amount, 2)
                hover_template = (f'Event: {label}<br>' +f'Token: {token}<br>' +f'Amount: ${amount}<br>' +'Date: %{x|%d %b %Y (%a)}<br>' +'Time: %{x|%H:%M:%S}<extra></extra>')
            else:
                hover_template = (f'Event: {label}<br>' +f'Token: {token}<br>' + 'Date: %{x|%d %b %Y (%a)}<br>' +'Time: %{x|%H:%M:%S}<extra></extra>')

            # Check if row is specified
            if row is not None:
                if row == 1:
                    temp_show_legend = show_legend
                else:
                    temp_show_legend = False
                fig.add_trace(go.Scatter(
                x=[datetime.fromtimestamp(ts), datetime.fromtimestamp(ts)],
                y=[0, 4],  # Use the range of the Health Factor
                mode='lines',
                line=dict(color=color, width=1),
                name=label,
                showlegend=temp_show_legend,
                legendgroup=label.lower(),  # Group under the same legend item
                hoverinfo='text',
                text = label,
                hovertemplate = hover_template), row=row, col=col, secondary_y=True)
                show_legend = False

            else:
                fig.add_trace(go.Scatter(
                x=[datetime.fromtimestamp(ts), datetime.fromtimestamp(ts)],
                y=[0, 4],  # Use the range of the Health Factor
                mode='lines',
                line=dict(color=color, width=1),
                name=label,
                showlegend=show_legend,
                legendgroup=label.lower(),  # Group under the same legend item
                hoverinfo='text',
                text = label,
                hovertemplate = hover_template), secondary_y=True)
                show_legend = False  # Set to False after adding the first line

    elif label == 'Liquidations':
        show_legend = True  # Only show legend for the first line
        for ts, collateral_asset_address, debt_asset_address, debt_to_cover, liquidated_coll_amount in event_timestamps:
            debt_asset = tokens[debt_asset_address]
            collateral_asset = tokens[collateral_asset_address]
            debt_to_cover = (debt_to_cover * get_usd_price(debt_asset_address, ts)) / (10**decimals[debt_asset])
            liquidated_coll_amount = (liquidated_coll_amount * get_usd_price(collateral_asset_address, ts)) / (10**decimals[collateral_asset])
            debt_to_cover = round(debt_to_cover, 2)
            liquidated_coll_amount = round(liquidated_coll_amount, 2)
            hover_template = (f'Event: {label}<br>' +f'Debt Asset: {debt_asset}<br>' +f'Amount of Debt Paid: ${debt_to_cover}<br>'+f'Collateral Asset: {collateral_asset}<br>' +f'Amount of Collateral Liquidated: ${liquidated_coll_amount}<br>' +'Date: %{x|%d %b %Y (%a)}<br>' +'Time: %{x|%H:%M:%S}<extra></extra>')

            # Check if row is specified
            if row is not None:
                if row == 1:
                    temp_show_legend = show_legend
                else:
                    temp_show_legend = False
                fig.add_trace(go.Scatter(
                x=[datetime.fromtimestamp(ts), datetime.fromtimestamp(ts)],
                y=[0, 4],  # Use the range of the Health Factor
                mode='lines',
                line=dict(color=color, width=1),
                name=label,
                showlegend=temp_show_legend,
                legendgroup=label.lower(),  # Group under the same legend item
                hoverinfo='text',
                text = label,
                hovertemplate = hover_template), row=row, col=col, secondary_y=True)
                show_legend = False

            else:
                fig.add_trace(go.Scatter(
                x=[datetime.fromtimestamp(ts), datetime.fromtimestamp(ts)],
                y=[0, 4],  # Use the range of the Health Factor
                mode='lines',
                line=dict(color=color, width=1),
                name=label,
                showlegend=show_legend,
                legendgroup=label.lower(),  # Group under the same legend item
                hoverinfo='text',
                text = label,
                hovertemplate = hover_template), secondary_y=True)
                show_legend = False  # Set to False after adding the first line

    elif label == 'Borrows':
        show_legend = True  # Only show legend for the first line
        for ts, reserve, amount, rate_mode, borrow_rate in event_timestamps:
            token = tokens[reserve]
            if label in ['Borrows']:
                amount = (amount * get_usd_price(reserve, ts)) / (10**decimals[token])
                borrow_rate = float(borrow_rate)
                borrow_rate /= (10**25)
                borrow_rate = round(borrow_rate, 2)
            amount = round(amount, 2)
            if int(rate_mode) == int(1):
                rate_mode = 'Stable'
            elif int(rate_mode) == int(2):
                rate_mode = 'Variable'

            # Check if row is specified
            if row is not None:
                if row == 1:
                    temp_show_legend = show_legend
                else:
                    temp_show_legend = False
                fig.add_trace(go.Scatter(
                x=[datetime.fromtimestamp(ts), datetime.fromtimestamp(ts)],
                y=[0, 4],  # Use the range of the Health Factor
                mode='lines',
                line=dict(color=color, width=1),
                name=label,
                showlegend=temp_show_legend,
                legendgroup=label.lower(),  # Group under the same legend item
                hoverinfo='text',
                text = label,
                hovertemplate = (f'Event: {label}<br>Token: {token}<br>Amount: ${amount}<br>Borrow Rate Mode: {rate_mode}<br>Borrow Rate: {borrow_rate}%<br>' +'Date: %{x|%d %b %Y (%a)}<br>' +'Time: %{x|%H:%M:%S}<extra></extra>')), secondary_y=True)
                show_legend = False

            else:
                fig.add_trace(go.Scatter(
                x=[datetime.fromtimestamp(ts), datetime.fromtimestamp(ts)],
                y=[0, 4],  # Use the range of the Health Factor
                mode='lines',
                line=dict(color=color, width=1),
                name=label,
                showlegend=show_legend,
                legendgroup=label.lower(),  # Group under the same legend item
                hoverinfo='text',
                text = label,
                hovertemplate = (f'Event: {label}<br>Token: {token}<br>Amount: ${amount}<br>Borrow Rate Mode: {rate_mode}<br>Borrow Rate: {borrow_rate}%<br>' +'Date: %{x|%d %b %Y (%a)}<br>' +'Time: %{x|%H:%M:%S}<extra></extra>')), secondary_y=True)
                show_legend = False  # Set to False after adding the first line

    else:
        show_legend = True  # Only show legend for the first line
        for ts in event_timestamps:
            # Check if row and col are specified
            if row is not None and col is not None:
                if row == 2:
                    temp_show_legend = False
                else:
                    temp_show_legend = show_legend
                fig.add_trace(go.Scatter(
                x=[datetime.fromtimestamp(ts), datetime.fromtimestamp(ts)],
                y=[0, 4],  # Use the range of the Health Factor
                mode='lines',
                line=dict(color=color, width=1),
                name=label,
                showlegend=temp_show_legend,
                legendgroup=label.lower(),  # Group under the same legend item
                hoverinfo='text',
                text = label,
                hovertemplate=f'Event: {label}' + '<br>Date: %{x|%d %b %Y (%a)}<br>Time: %{x|%H:%M:%S}<extra></extra>'), row=row, col=col, secondary_y=True)

            else:
                fig.add_trace(go.Scatter(
                x=[datetime.fromtimestamp(ts), datetime.fromtimestamp(ts)],
                y=[0, 4],  # Use the range of the Health Factor
                mode='lines',
                line=dict(color=color, width=1),
                name=label,
                showlegend=show_legend,
                legendgroup=label.lower(),  # Group under the same legend item
                hoverinfo='text',
                text = label,
                hovertemplate=f'Event: {label}' + '<br>Date: %{x|%d %b %Y (%a)}<br>Time: %{x|%H:%M:%S}<extra></extra>'
            ), secondary_y=True)

            show_legend = False  # Set to False after adding the first line


In [133]:
for i in range(len(user_df)):
    user = user_df.borrower.iloc[i]
    temp_df = pd.read_csv(f"/content/drive/MyDrive/aave_files/aave_user_details_v2/{user}_details_v2.csv")
    temp_df['dt'] = temp_df['timestamp'].apply( lambda x: datetime.fromtimestamp(x) )

    transfers_in_df = transfers_df[transfers_df['to'] == user]
    transfers_in = transfers_in_df['timestamp'].to_list()
    transfers_in_detail = list(zip(transfers_in_df['timestamp'].to_list(), transfers_in_df['reserve'].to_list(), transfers_in_df['value (in USD)'].to_list()))

    transfers_out_df = transfers_df[transfers_df['from'] == user]
    transfers_out = transfers_out_df['timestamp'].to_list()
    transfers_out_detail = list(zip(transfers_out_df['timestamp'].to_list(), transfers_out_df['reserve'].to_list(), transfers_out_df['value (in USD)'].to_list()))

    events_lending_pool_df = new_lending_pool_df[(new_lending_pool_df['user'] == user) | (new_lending_pool_df['onBehalfOf'] == user) | (new_lending_pool_df['from'] == user) | (new_lending_pool_df['to'] == user)]

    liq_lending_pool_df = lending_pool_df[(lending_pool_df['user'] == user) & (lending_pool_df['event'] == 'LiquidationCall')]
    liqs = liq_lending_pool_df['timestamp'].to_list()
    liqs_detail = list(zip(liq_lending_pool_df['timestamp'].to_list(), liq_lending_pool_df['collateralAsset'].to_list(), liq_lending_pool_df['debtAsset'].to_list(), liq_lending_pool_df['debtToCover'].to_list(), liq_lending_pool_df['liquidatedCollateralAmount'].to_list()))

    withdraw_lending_pool_df = lending_pool_df[(lending_pool_df['user'] == user) & (lending_pool_df['event'] == 'Withdraw')]
    withdraws = withdraw_lending_pool_df['timestamp'].to_list()
    withdraws_detail = list(zip(withdraw_lending_pool_df['timestamp'].to_list(), withdraw_lending_pool_df['reserve'].to_list(), withdraw_lending_pool_df['amount'].to_list()))

    repay_lending_pool_df = lending_pool_df[((lending_pool_df['user'] == user ) | (lending_pool_df['onBehalfOf'] == user)) & (lending_pool_df['event'] == 'Repay')]
    repays = repay_lending_pool_df['timestamp'].to_list()
    repays_detail = list(zip(repay_lending_pool_df['timestamp'].to_list(), repay_lending_pool_df['reserve'].to_list(), repay_lending_pool_df['amount'].to_list()))

    borrow_lending_pool_df = lending_pool_df[((lending_pool_df['user'] == user ) | (lending_pool_df['onBehalfOf'] == user)) & (lending_pool_df['event'] == 'Borrow')]
    borrows = borrow_lending_pool_df['timestamp'].to_list()
    borrows_detail = list(zip(borrow_lending_pool_df['timestamp'].to_list(), borrow_lending_pool_df['reserve'].to_list(), borrow_lending_pool_df['amount'].to_list(), borrow_lending_pool_df['borrowRateMode'].to_list(), borrow_lending_pool_df['borrowRate'].to_list()))

    deposit_lending_pool_df = lending_pool_df[((lending_pool_df['user'] == user ) | (lending_pool_df['onBehalfOf'] == user)) & (lending_pool_df['event'] == 'Deposit')]
    deposits = deposit_lending_pool_df['timestamp'].to_list()
    deposits_detail = list(zip(deposit_lending_pool_df['timestamp'].to_list(), deposit_lending_pool_df['reserve'].to_list(), deposit_lending_pool_df['amount'].to_list()))

    coll_enab_lending_pool_df = lending_pool_df[(lending_pool_df['user'] == user) & (lending_pool_df['event'] == 'ReserveUsedAsCollateralEnabled')]
    coll_enabs = coll_enab_lending_pool_df['timestamp'].to_list()
    coll_enabs_detail = list(zip(coll_enab_lending_pool_df['timestamp'].to_list(), coll_enab_lending_pool_df['reserve'].to_list(), coll_enab_lending_pool_df['amount'].to_list()))

    coll_disab_lending_pool_df = lending_pool_df[(lending_pool_df['user'] == user) & (lending_pool_df['event'] == 'ReserveUsedAsCollateralDisabled')]
    coll_disabs = coll_disab_lending_pool_df['timestamp'].to_list()
    coll_disabs_detail = list(zip(coll_disab_lending_pool_df['timestamp'].to_list(), coll_disab_lending_pool_df['reserve'].to_list(), coll_disab_lending_pool_df['amount'].to_list()))

    event_details = [liqs_detail, withdraws_detail, borrows_detail, repays_detail, deposits_detail, coll_enabs_detail, coll_disabs_detail, transfers_in_detail, transfers_out_detail]
    events = ['Liquidations', 'Withdrawals', 'Borrows', 'Repays', 'Deposits', 'Reserve Enabled as Collateral', 'Reserve Disabled as Collateral', 'Transfer In', 'Transfer Out']
    event_colors = ['blue', 'red', 'green', 'orange', 'black', 'violet', 'brown', 'lime', 'aqua']

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    for event_detail, event, event_color in list(zip(event_details, events, event_colors)):
        add_event_lines(fig, event_detail, event_color, event)

    fig.add_trace(go.Scatter(x=temp_df['dt'], y=temp_df['totalCollateral (in USD)'], mode='lines', name='Total Collateral (in USD)', hovertemplate='Total Collateral Amount: $%{y}<br>Date: %{x|%d %b %Y (%a)}<br>Time: %{x|%H:%M:%S}<extra></extra>'), secondary_y=False)
    fig.add_trace(go.Scatter(x=temp_df['dt'], y=temp_df['totalDebt (in USD)'], mode='lines', name='Total Debt (in USD)', hovertemplate='Total Debt Amount: $%{y}<br>Date: %{x|%d %b %Y (%a)}<br>Time: %{x|%H:%M:%S}<extra></extra>'), secondary_y=False)
    fig.add_trace(go.Scatter(x=temp_df['dt'], y=temp_df['healthFactor'], mode='lines', name='Health Factor', hovertemplate='Health Factor: %{y}<br>Date: %{x|%d %b %Y (%a)}<br>Time: %{x|%H:%M:%S}<extra></extra>', line=dict(color='black')), secondary_y=True)

    # Add horizontal line at Health Factor 1
    fig.add_hline(y=1, line=dict(color='red', width=2, dash='dash'), secondary_y=True)

    fig.update_layout(
        title=f"{user} Total Collateral & Total Debt with Health Factor",
        xaxis_title='Date',
        yaxis_title='Total Collateral and Debt amounts (in USD)',
        yaxis2_title='Health Factor',
        xaxis=dict(tickformat='%d %b %Y'),  # Set x-axis tick format
        legend=dict(font=dict(size=7), tracegroupgap=1)
    )

    fig.show()

    combined_fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, specs=[[{"secondary_y": True}], [{"secondary_y": True}]])

    for col in temp_df.columns:
        if '(in USD)' in col and not 'total' in col:
            symb = (col.split()[0]).split('_')[0]
            symb_type = ((col.split()[0]).split('_')[1]).title()
            combined_fig.add_trace(go.Scatter(x=temp_df['dt'], y=temp_df[col], mode='lines', name=col, hovertemplate=f'{symb} {symb_type}<br>Amount: ' + '$%{y}<br>Date: %{x|%d %b %Y (%a)}<br>Time: %{x|%H:%M:%S}<extra></extra>'),  row = 1, col = 1, secondary_y = False)
    combined_fig.add_trace(go.Scatter(x=temp_df['dt'], y=temp_df['healthFactor'], mode='lines', name='Health Factor', hovertemplate='Health Factor: %{y}<br>Date: %{x|%d %b %Y (%a)}<br>Time: %{x|%H:%M:%S}<extra></extra>', line=dict(color='black')), row = 1, col = 1, secondary_y=True)
    combined_fig.add_hline(y=1, line=dict(color='red', width=2, dash='dash'), row = 1, col = 1, secondary_y=True)

    for col in temp_df.columns:
        if 'WETH_collateral (in USD)' in temp_df.columns or 'WETH_debt (in USD)' in temp_df.columns:
            if 'USD_price' in col:
                symb = (col.split()[0]).split('_')[0]
                combined_fig.add_trace(go.Scatter(x=temp_df['dt'], y=temp_df[col],mode='lines', name=col, hovertemplate=f'{symb} <br>Price: ' + '$%{y}<br>Date: %{x|%d %b %Y (%a)}<br>Time: %{x|%H:%M:%S}<extra></extra>'),  row = 2, col = 1,secondary_y = False)
        else:
            #Not adding WETH price when there's no WETH Collateral or debt.
            if 'USD_price' in col and 'WETH' not in col:
                symb = (col.split()[0]).split('_')[0]
                combined_fig.add_trace(go.Scatter(x=temp_df['dt'], y=temp_df[col], mode='lines', name=col, hovertemplate=f'{symb} <br>Price: ' + '$%{y}<br>Date: %{x|%d %b %Y (%a)}<br>Time: %{x|%H:%M:%S}<extra></extra>'),  row = 2, col = 1,secondary_y = False)

    for row in [1,2]:
        for event_detail, event, event_color in list(zip(event_details, events, event_colors)):
            add_event_lines(combined_fig, event_detail, event_color, event, row, 1)

    # Step 4: Adjust the layout and axis titles
    combined_fig.update_layout(
        title=f"{user} Individual Collaterals and Debt Amounts and Token Prices (in USD)",
        # xaxis_title='Date',
        yaxis_title='Amount (in USD)',
        yaxis2_title='Health Factor',
        legend=dict(font=dict(size=7), tracegroupgap=1)
    )
    combined_fig.update_xaxes(title_text="Date", row=2, col=1, tickformat='%d %b %Y')
    combined_fig.update_yaxes(title_text="Collateral/Debt Amounts", row=1, col=1, secondary_y=False)
    combined_fig.update_yaxes(title_text="Token Prices", row=2, col=1, secondary_y=False)
    combined_fig.update_yaxes(title_text="Health Factor", row=1, col=1, secondary_y=True)

    combined_fig.show()